In [0]:
%sql
USE CATALOG nyc_taxi;
USE SCHEMA idk;

In [0]:
df = spark.table("nyc_taxi.idk.yellow_trips_csv_v")
df.createOrReplaceTempView("table")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW trips_enriched AS
SELECT
  *,
  DATE(tpep_pickup_datetime) AS trip_date,                
  HOUR(tpep_pickup_datetime) AS pickup_hour,             
  (unix_timestamp(tpep_dropoff_datetime) -
   unix_timestamp(tpep_pickup_datetime)) / 60.0 AS duration_min  
FROM table
WHERE tpep_pickup_datetime IS NOT NULL
  AND tpep_dropoff_datetime IS NOT NULL;

  --helper view to reduce calculations for questions 8-12


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW revenue_by_weekday AS
SELECT
  DATE_FORMAT(tpep_pickup_datetime, 'E') AS day_of_week,
  SUM(total_amount)                      AS total_revenue,
  CASE DATE_FORMAT(tpep_pickup_datetime, 'E')
    WHEN 'Mon' THEN 1 WHEN 'Tue' THEN 2 WHEN 'Wed' THEN 3
    WHEN 'Thu' THEN 4 WHEN 'Fri' THEN 5 WHEN 'Sat' THEN 6
    WHEN 'Sun' THEN 7
  END AS day_order
FROM table
GROUP BY day_of_week, day_order
ORDER BY day_order;

SELECT day_of_week, total_revenue FROM revenue_by_weekday;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW busiest_hour AS
SELECT
  pickup_hour,
  COUNT(*) AS pickups
FROM trips_enriched
GROUP BY pickup_hour
ORDER BY pickups DESC;

SELECT * FROM busiest_hour;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW avg_duration_by_hour AS
SELECT
  pickup_hour,
  AVG(duration_min) AS avg_duration_min
FROM trips_enriched
GROUP BY pickup_hour
ORDER BY pickup_hour;

SELECT * FROM avg_duration_by_hour;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW avg_fare_by_payment AS
SELECT
  payment_type,
  AVG(fare_amount) AS avg_fare
FROM table
WHERE fare_amount IS NOT NULL
GROUP BY payment_type
ORDER BY payment_type;

SELECT * FROM avg_fare_by_payment;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW trips_per_day AS
SELECT
  trip_date,
  COUNT(*) AS trip_count
FROM trips_enriched
GROUP BY trip_date
ORDER BY trip_date;

SELECT * FROM trips_per_day;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW top10_pickups AS
SELECT
  pickup_longitude,
  pickup_latitude,
  COUNT(*) AS trips
FROM table
WHERE pickup_longitude IS NOT NULL AND pickup_latitude IS NOT NULL
GROUP BY pickup_longitude, pickup_latitude
ORDER BY trips DESC
LIMIT 10;

SELECT * FROM top10_pickups;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW top10_dropoffs AS
SELECT
  dropoff_longitude,
  dropoff_latitude,
  COUNT(*) AS trips
FROM table
WHERE dropoff_longitude IS NOT NULL AND dropoff_latitude IS NOT NULL
GROUP BY dropoff_longitude, dropoff_latitude
ORDER BY trips DESC
LIMIT 10;

SELECT * FROM top10_dropoffs;
